# Chapter 12 - Custom Models and Training with Tensorflow

<center><img src="img/tfAPI.png"></img></center>
<center><img src="img/tfAPI2.png"></img></center>

### Using tensorflow like numpy

In [2]:
import tensorflow as tf
# tensorflow operations
m = tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix
m

2022-01-07 18:41:22.259511: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
c = tf.constant(42)
c

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [6]:
print(m.shape)
print(m.dtype)

(2, 3)
<dtype: 'float32'>


In [8]:
# Indexing
m[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [14]:
m[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [15]:
m + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [16]:
tf.square(m)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [19]:
# @ is the matrix multiplication operator, equivalent to tf.matmul() 
m @ tf.transpose(m)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

Keras low level API is limited (keras.backend), there are only a few operations available, so we stick with Tensorflow

In [20]:
from tensorflow import keras
K = keras.backend
K.square(K.transpose(m)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### Tensors and Numpy
They work well together, many things in common

In [21]:
import numpy as np
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [25]:
m.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [26]:
# 32-bit is more than enough, change it to use less RAM
a2 = tf.constant(a, dtype=tf.float32)
a2

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 4., 5.], dtype=float32)>

### Type conversions
TensorFlow does not allow automatic conversions

In [27]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [28]:
# Not even with different byte representation
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]

In [29]:
# Using cast
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Variables
_tf.Tensor_ is immutable, they can't be weights in a DNN or other parameters, we use tf.Variable in this case. In practice we will rarely create variables manually, since Keras provides an add_weight() method and the model parameter will be updated by the optimizers.

In [33]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [34]:
# using assign methods with tf.Variable
print(v.assign(2 * v))
print('\n', v[0, 1].assign(42))
print('\n', v[:, 2].assign([0., 1.]))
print('\n', v.scatter_nd_update(indices=[[0,0], [1, 2]], updates=[100., 200.]))

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

 <tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

 <tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

 <tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>


### Oher Data Structures
- _Sparse tensors_ (tf.SparseTensor): Efficient represent tensors containing mostly zeros. The tf.sparse package contains operations for sparse tensors.
- _Tensor arrays_ (tf.TensorArray) : List of tensors, fixed size by default but can be made dynamic. All tensors must have same shape and data type.
- _Ragged tensors_ (tf.RaggedTensor): static lists of lists of tensors, same shape and type. Use tf.ragged for operations.
- _String tensors_: represent byte strings, not unicode. A Python string is converted to UTF-8. Another option is to use tf
.int32 where each item represents an Unicode code point
- _Sets_: Regular tensors (sparse tensors). tf.constant([[1,2], [3,4]]) represent 2 sets {1,2}, {3,4}
- _Queues_: Store tensors across multiple steps, there are many kinds, First In, First Out (FIFO) queues (FIFOQueue), queues that can prioritize some items (PriorityQueue), shuffle their items (RandomSheffleQueue) and batch items of different shapes by padding (PaddingFIFOQueue). This classes are un the tf.queue package.

Now we are ready to start customizing models and training algorithms!